# Analysis of Poetry

The cells below firstly reads in one of the texts in the folder 'Poems'

In [ ]:
## If this is the first time you work with nltk
## You probably need to install a number of 
## additional modules and packages

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [ ]:
import poetry_analysis as pa
import os
import re

dir = 'Poems'
file = 'NO SECOND TROY.txt'

full_path = os.path.join( dir, file )

with open(full_path, encoding = 'utf-8') as fh:
    for line in fh:
        print(line.strip())

## Annotations

The module `poetry_analysis` contains a method named `add_annotation`to add a POS tag, a lemma and a phonetic transcription. The annotations are added in the form of TEI tags. 

In [ ]:
tei = pa.add_annotations(full_path)

print('\n')
print(tei)

We can add such annotations for all the texts in the folder 'Poems'. The code below saves all the results in a folder named 'XML'. 

In [ ]:
def encodeFileName(text):
    text = re.sub( r'[\'`]' , '&#x27;' , text)
    text = re.sub( r'\s+' , '_' , text)
    return text

if not os.path.isdir('XML'):
    os.mkdir('XML')
    
dir = 'Poems'

texts = []

for root, dirs, files in os.walk(dir):
    for file in files:
        if re.search( r'\.txt$' , file ):
            texts.append( os.path.join(root, file) )

for t in texts:
    print(t)
    
    out_file = os.path.basename(t)
    out_file = re.sub( 'txt$' , 'xml' , out_file )
    out_file = encodeFileName(out_file)
    out = open( os.path.join( 'XML' , out_file ) , 'w' , encoding='utf-8' )
    
    tei = pa.add_annotations(t)
    out.write(tei)
    out.close()
    
print('Done!')
    
  

The data in these TEI files can be examined a number of different methods in the module `poetry_analysis`.

In [ ]:
dir = 'XML'
file = 'LEDA_AND_THE_SWAN.xml'
path = os.path.join( dir, file )

poem = pa.poem( path )

print( f'Number of lines: {poem.nr_lines} ' )
print( f'Number of words: {poem.nr_words} ' )
print( f'Number of stanzas: {poem.nr_stanzas} ' )

We can print all the phonetic transcriptions (in SAMPA notation).

In [ ]:
transcr = poem.transcriptions
lines = poem.lines

for n in lines:
    print( lines[n] )
    print( transcr[n] )
    print('\n')

## Alliteration

In [ ]:
dir = 'XML'

for tei in os.listdir(dir):
    print( f'\n{tei}')
    path = os.path.join( dir, tei )

    poem = pa.poem( path )

    transcr = poem.transcriptions
    lines = poem.lines

    for n in transcr:
        alliteration = pa.alliteration(transcr[n])

        if re.search( r'\w' , alliteration ):
            print(lines[n])
            print( alliteration )
            print( '\n')
           
    

# Internal rhyme

In [ ]:
dir = 'XML'
tei = 'Budgie.xml'

print( f'\n{tei}')
path = os.path.join( dir, tei )
poem = pa.poem( path )

transcr = poem.transcriptions
lines = poem.lines

for n in transcr:
    internal_rhyme = pa.internal_rhyme(transcr[n])
    if len(internal_rhyme) > 0:
        print(lines[n] + ' => ' + ','.join(internal_rhyme)  )

# Perfect rhyme

In [ ]:

def perfect_rhyme( tei):
    
    print(tei + '\n')

    path = os.path.join( dir, tei )
    poem = pa.poem( path )

    lines = poem.lines
    tr = poem.transcriptions

    stanzas = poem.stanza_structure
    for s in stanzas:
        stanza_lines = []
        for n in stanzas[s]:
            stanza_lines.append(tr[n])

        pr = pa.perfect_rhyme(stanza_lines)
        if re.search( r'\d' , pr ):
            for n in stanzas[s]:
                print(lines[n])
            print(pr)

            
perfect_rhyme('DOWN_BY_THE_SALLEY_GARDENS.xml')
perfect_rhyme('THE_STOLEN_CHILD.xml')
perfect_rhyme('SUNDAY_MORNING.xml')



In [ ]:
import os
import re
import poetry_analysis as pa

out = open( 'data.csv', 'w' , encoding = 'utf-8' )


columns = [ 'title' , 
           'nr_words' , 
           'nr_stanzas' , 
           'nr_lines' , 
           'alliteration' , 
           'internal_rhyme' , 
           'perfect_rhyme' ]

header = ','.join(columns)
out.write( header )
out.write('\n')

for file in os.listdir('XML'):
    if re.search( 'xml$' , file ):
        print( f'Collecting data for {file} ...')

        data = dict()
        data['title'] = file[ 0 : file.index('.') ].title()   
        poem = pa.poem(os.path.join( 'XML' , file))
        data['nr_words'] = poem.nr_words
        data['nr_stanzas'] = poem.nr_stanzas
        data['nr_lines'] = poem.nr_lines

        tr = poem.transcriptions
        alliteration_count = 0
        internal_rhyme_count = 0 

        for line in tr:
            alliteration = pa.alliteration(tr[line])
            if re.search( r'\w' , alliteration ):
                alliteration = re.sub('-|\s+', '' , alliteration )
                alliteration_count += len( alliteration )

            internal_rhyme = pa.internal_rhyme(tr[line])
            if len(internal_rhyme) > 0:
                internal_rhyme_count += len(internal_rhyme)

        data['alliteration'] = alliteration_count
        data['internal_rhyme'] = internal_rhyme_count

        stanzas = poem.stanza_structure
        pr_count = 0

        for s in stanzas:
            stanza_lines = []
            for n in stanzas[s]:
                stanza_lines.append(tr[n])

            ## perfect rhyme
            pr = pa.perfect_rhyme(stanza_lines)
            if re.search( r'\d' , pr ):
                pr = re.sub('-|\s+', '' , pr )
                pr_count += len( pr )

        data['perfect_rhyme'] = pr_count



        for i,c in enumerate(columns):
            out.write( f'{data[c]}')
            if i == len(columns)-1:
                out.write('\n')
            else:
                out.write(',')

out.close() 
 